Document extracter

In [65]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("kafka.pdf")
pages = loader.load()

In [66]:
from langchain_core.documents import Document
import os
os.makedirs("../data/text_files",exist_ok=True)

txtDir = "../data/text_files"

In [67]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """
}


In [70]:

#save text into files
def save_text(filepath, content):
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(content)
        print(f"Saved: {filepath}")


text = "".join(page.page_content for page in pages[100:150])
text = preprocess_text(text)

save_text(os.path.join(txtDir, "kf100.txt"), text)

# for filepath, content in sample_texts.items():
#     save_text(filepath, content)

Saved: ../data/text_files\kf100.txt


In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("../data/text_files/python_intro.txt")
document = loader.load()


In [19]:
def preprocess_text(text):
    text = text.replace('\t', ' ')
    text = text.replace('\n', ' ')
    return text

text = preprocess_text(text)
tex = text[0:500]
tex

'first listened to him—it bored me silly. It\'s only natural for someone your age. In time you\'ll appreciate it. People soon get tired of things that aren\'t boring, but not of what is boring. Go figure. For me, I might have the leisure to be bored, but not to grow tired of something. Most people can\'t distinguish between the two."   "You said you\'re an unusual person. Do you mean because of the hemophilia?"   "That\'s part of it," he says, and gives this devilish sort of smile. "There\'s more to it '

In [48]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=50,
    length_function=len,
)
c_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len)


In [34]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()
# 3.1 Initialize the Embedding Model
# NOTE: The model and task type are crucial for retrieval accuracy.
embedding = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",  # A strong general-purpose model
    task_type="retrieval_document" # Optimizes the embedding for document search
)



## Embeddings
The dot product of the embedded vectors shows the similarity between the words. 
Embeddings are created using a pretrained models. These models can be decided based on the use case of the similarity. we have bunch of models like openai , gemini embedding, hugging face also hasd some 

In [36]:
import numpy as np

sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

print(np.dot(embedding1, embedding2))  # Similarity between sentence1 and sentence2
print(np.dot(embedding1, embedding3))  # Similarity between sentence1 and sentence  3

0.972301896586312
0.7972381759029037


In [ ]:
chroma_db = '../data/chroma/'

In [71]:
# load texts 
from langchain.document_loaders import TextLoader
import os
    
loader = TextLoader("../data/text_files/kf100.txt")
texts = loader.load()

In [75]:
# text = texts[0].page_content
# t1 = preprocess_text(text)
chunks = r_splitter.split_documents(texts)
len(chunks)
# chunks

418

In [57]:
!rm -rf chroma_db

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [83]:
# The `from_documents` method automatically handles the embedding and storage.
vector_store = Chroma.from_documents(
    documents=chunks, 
    embedding=embedding, 
    persist_directory=chroma_db
)

In [85]:
embedding = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",  # A strong general-purpose model
    task_type="retrieval_document" # Optimizes the embedding for document search
)
v1 = Chroma(
    persist_directory=chroma_db,
    embedding_function=embedding
)
print(v1._collection.count())

C:\Users\shahn\AppData\Local\Temp\ipykernel_15968\2976856195.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  v1 = Chroma(


418


In [ ]:
! pip install -U :class:`~langchain-chroma
! 

In [91]:
question = "what did they say about kafka?"
docs_ss = v1.similarity_search(question,k=6)
print(len(docs_ss))
print(docs_ss[0].page_content)
print(docs_ss[1].page_content)
print(docs_ss[2].page_content)

6
got that impression."   I don't reply.   "From my own experience, when someone is trying very hard to get something, they don't. And when they're running away from something as hard as they can, it usually catches up with them. I'm generalizing, of course."   "If you generalize about me, then, what's in my future? If I'm seeking and running at the
first listened to himâ€”it bored me silly. It's only natural for someone your age. In time you'll appreciate it. People soon get tired of things that aren't boring, but not of what is boring. Go figure. For me, I might have the leisure to be bored, but not to grow tired of something. Most people can't distinguish between the two."   "You said
on the record jacket of "Kafka on the Shore." But there's one thing missing: that lovely, innocent smile. She still smiles from time to time, definitely a charming smile, but it's always limited somehow, a smile that never goes beyond the moment. A high, invisible wall surrounds her, holding people at 